## Notebook of demos, tutorial

by Darach, @darachm

This notebook is intended to give new users an idea of how `itermae` works to 
accomplish basic parsing, how to debug its usage, how to run it in parallel, 
and how to use multiple matches to do more complex parsing and filtering.

It is intended to be run in the `itermae` repository, so it can access the 
example data and config files in that repo. Uses the `bash_kernel`, of course.

Any commentary or feedback on this tool or documentation would be very valuable 
and very welcome, so please reach out if you have the time to do so.
Submit an [issue on Gitlab](https://gitlab.com/darachm/itermae/-/issues).

## Table of contents:

1. [The idea](#idea)
1. [Installation](#install)
1. [Quickstart](#quickstart)
1. [Detailed usage](#usage)

    1. [Input/output, formats and failed/report files](#io)
    1. [Matches and how to write](#matches)
    1. [Outputs, filtering and constructing](#outputs)

2. [Debugging](#operations)
3. Parallel chopping, intended usage
4. Multiple operations

## The idea <a id="idea" />

We'd like a robust tool to parse DNA amplicon-sequencing of 
datasets that use complex DNA barcoding architectures.
Often, positional information is used to extract these. This can
frequently fail in extracting the wrong sequence due to random
or systematic deviation from the expectation --
DNA sequence (especially degenerately
synthesized/cloned sequence) is mutable both due to technical
errors (in PCR and sequencing) or biological mutagenesis. 
Both variable lengths of barcodes or technical 
insertion/deletion errors can throw an entire read out of frame,
hence regular-expression based methods can be used to extract 
patterns from reads. However, these are brittle to substitition
errors in the "fixed" sequences if error-tolerant (ie 'fuzzy')
regular expressions are not used. 
These can result in artifactual matches, such pattern matches in 
very different positions of the read, unless these are composed
with filtering steps and/or multiple matching operations to make
higher-level comparisons.
But, generating a single large fuzzy regular expression is 
difficult, and complex filtering or look-back/forward expressions 
may be difficult for non-expert users to write, debug, and update.

The aim of the tool is to allow a user to flexibily apply multiple
fuzzy regular-expressions to a sequence-record, in series on the
same read, and then flexibily filter and build outputs based on 
what matches. For example, here is one FASTQ read with relevant
patterns extracted and re-arranged for downstream use, all using
fuzzy regular expressions.

There are three main groups of options to configure:

- Input/Output logistics : what formats to read and write, and to where
- Matches : what matches to apply, to what groups, what groups to 
  capture, and how error-tolerant to be
- Outputs : how to filter reads and what outputs to construct from
  matched groups
  
Below I detail what options are available, how to specify them,
and a recommended strategy to tune the options and use the tool.

## Install

Install `itermae` with pip.

In [ ]:
python3 -m pip install itermae

You can also build and install the package from inside the repository, by using 
the `make install` command. 

Package requirements (beyond base Python >= 3.6) are listed in
`requirements.txt`, including `pytest` for development and testing.

I also recommend installing [GNU `parallel`](https://www.gnu.org/software/parallel/),
as this utility allows you to easily parallelize text-based file inputs. 
`itermae` was designed for the use-case of a sequencing input file being fed by
`parallel` through a bash pipe into the STDIN of `itermae`.

I recommend you [install parallel from source](https://ftp.gnu.org/gnu/parallel/) 
or use the [Singularity image for itermae](https://singularity-hub.org/collections/4537), 
as some package managers (ie ubuntu) are woefully unmaintained.

## Quickstart <a id="quickstart" />

Copy and modify the below command. 

- You can change the `-N 1000` to reflect how many records to 
  feed into the script at a time.
  
- Edit the config file to specify the input/output formats,
  the matches executed, and how the outputs are filtered and
  formed from these matches.
  
More options, and `parallel` crash-course usage, are detailed later.

In [48]:
zcat itermae/data/barseq.fastq.gz \
    | parallel --pipe -l 4 -N 1000 \
        itermae --config itermae/data/example_schema.yml \
    | head

NB501157:100:H5J5LBGX2:1:11101:10000:10043	0	*	0	255	*	=	0	0	TCAGTCGTAGCAGTTCGATG	EEEEEEEEEEEEEEEEEEEE	IE:Z:0
NB501157:100:H5J5LBGX2:1:11101:10000:10043	0	*	0	255	*	=	0	0	TTCAC	AAAAA	IE:Z:1
NB501157:100:H5J5LBGX2:1:11101:10000:10043_TTCAC	0	*	0	255	*	=	0	0	TTCACXTCAGTXTCAGTCGTAGCAGTTCGATG	AAAAAIEEEEEIEEEEEEEEEEEEEEEEEEEE	IE:Z:2
NB501157:100:H5J5LBGX2:1:11101:10000:10138	0	*	0	255	*	=	0	0	TGGGCAGACACAACGCTACA	EEEEEEEEEEEEEEEEEEEE	IE:Z:0
NB501157:100:H5J5LBGX2:1:11101:10000:10138	0	*	0	255	*	=	0	0	GCTTC	AAAAA	IE:Z:1
NB501157:100:H5J5LBGX2:1:11101:10000:10138_GCTTC	0	*	0	255	*	=	0	0	GCTTCXTGGGCXTGGGCAGACACAACGCTACA	AAAAAIEEEEEIEEEEEEEEEEEEEEEEEEEE	IE:Z:2
NB501157:100:H5J5LBGX2:1:11101:10000:16613	0	*	0	255	*	=	0	0	GACAGACTGATAACCCTTGC	EEEEEEEEEEEEEEEEEEEE	IE:Z:0
NB501157:100:H5J5LBGX2:1:11101:10000:16613	0	*	0	255	*	=	0	0	GCTTC	AAAAA	IE:Z:1
NB501157:100:H5J5LBGX2:1:11101:10000:16613_GCTTC	0	*	0	255	*	=	0	0	GCTTCXGACAGXGACAGACTGATAACCCTTGC	AAAAAIEEEEEIEEEEEEEEEEEEEEEEEEEE	IE:Z:2
NB501157:1

At minimum, you probably need to know how to use:

* i/o
* matches
* output

## Detailed usage <a id="usage" />

Everything can be configured on command-line arguments or in a
YAML config file.

Here's an example with command-line arguments, which requires writing
raw regular expressions:

In [54]:
zcat itermae/data/barseq.fastq.gz | head -n 8 \
    | itermae -m 'input > (?P<sampleIndex>[ATCGN]{5,5})(?P<prefix>GTCCACGAGGTCTCT){e<=2}(?P<barcode>[ATCGN]{18,22})(?P<untitled_1>CGTACGCTG){i<=1,d<=2,s<=2}' \
            -os 'barcode' -oi 'input.id+"_"+sampleIndex.seq' \
            -of 'barcode.length >= 17 and sampleIndex.length >= 3' \
            --output-format fasta 


gzip: stdout: Broken pipe
>NB501157:100:H5J5LBGX2:1:11101:10000:10043_TTCAC
TCAGTCGTAGCAGTTCGATG
>NB501157:100:H5J5LBGX2:1:11101:10000:10138_GCTTC
TGGGCAGACACAACGCTACA


Alternatively, you can specify a YAML/YML configuration file:

In [56]:
zcat itermae/data/barseq.fastq.gz | head -n 8 \
    | itermae --config itermae/data/tutorial_example.yml


gzip: stdout: Broken pipe
>NB501157:100:H5J5LBGX2:1:11101:10000:10043_TTCAC
TCAGTCGTAGCAGTTCGATG
>NB501157:100:H5J5LBGX2:1:11101:10000:10138_GCTTC
TGGGCAGACACAACGCTACA


This is simpler, although it requires specifying a more bulky
configuration file, like so:

```
matches:
    -   use: input
        pattern: NGTCCACGAGGTCTCTNCGTACGCTG 
        marking: ABBBBBBBBBBBBBBBCDDDDDDDDD 
        marked_groups:
            A:
                name: sampleIndex
                repeat: 5
            B:
                name: prefix
                allowed_errors: 2
            C:
                name: barcode
                repeat_min: 18
                repeat_max: 22
            D:  
                allowed_insertions: 1
                allowed_deletions: 2
                allowed_substititions: 2

output: 
    format: fasta 
    list: 
        -   name: output
            filter: 'barcode.length >= 17 and sampleIndex.length >= 3'
            id:  'input.id+"_"+sampleIndex.seq'
            seq: 'barcode'
```

Below are details of the options available and how to specify them in both formats.
The notation

     [...]

denotes various other arguments necessary to run that I have left out so that
I can just point your attention to the relevant argument.

### Input and output options <a id="io"/>

#### What format are the input records in, and what format to write outputs in?

`itermae` makes no attempt to auto-detect formats.
The default is 'fastq' for input and 'sam' for output.
This tool handles:
  
  - 'FASTQ' files.
  - 'FASTA' files, where per-base quality scores are assumed to be the
      maximum value.
  - 'SAM' files. For input, all flags are ignored and not included in the 
      outputs. Similarlly, all alignment information is forgotten.
  - 'TXT files, where there is a single sequence per line. This ID of
      the record is defined by the sequence, and all qualities are 
      assumed to be the maximum value.
  - Any other format that BioPython SeqIO will recognize.
  
The format string for specifying is case-insensitive.
    
Specify with the command-line argument `--input-format`, `--output-format`,
or set the YAML key `format:` inside the `input:` or `output:` blocks. Ex:

    itermae --input-format sam --output-format fasta [...]

or in a config file

    input:
        format: sam
    output:
        format: fasta

#### Input source - where are the input reads coming from?

The default is 'STDIN', which specifies to expect reads to be piped ("`|`")
in to the tool. Alternatively you can specify a file path to open.
    
Specify with the command-line argument `--input` or `-i`, or set the YAML 
key `from:` inside the `input:` block. Ex:

    cat itermae/data/barseq.fastq | itermae [...]
    
to use the default STDIN, or

    itermae --input itermae/data/barseq.fastq [...]
or

    itermae -i itermae/data/barseq.sam --input-format sam [...]
    
or in a config file

    from:
        input: itermae/data/barseq.fasta
        format: fasta

#### Output target - where to send the successfully formed and  passing-filter outputs?

The default is 'STDOUT' to then be compressed, written, or further processed.
Alternatively you can specify a file path to open.
Messages from the `--verbose` command are output to 'STDERR' to not conflict
with 'STDOUT'.
  
Specify with the command-line argument `--output` or `-o`, or set the YAML 
key `to:` inside the `output:` block. Ex:

    itermae [...] > itermae_output_file.sam

to use the default STDOUT, or

    itermae --output itermae_output_file.fasta --output-format fasta [...]

or in a config file

    output:
        to: itermae_output_file.fastq
        format: fastq

#### Is the input file gzip compressed?

The default is no. Files specified can be gzipped, but of course the
default 'STDIN' cannot be piped in as compressed (pass through `zcat`).
   
Specify with the command-line argument `--gzipped` or `-z`, 
or set the YAML key `gzipped:` inside the `input:` block to 
`true` or `false`. Ex:

    itermae --input itermae/data/barseq.fastqz --gzipped [...] 

or

    zcat itermae/data/barseq.fastqz | itermae [...] 
    
or in a config file

    from:
        input: itermae/data/barseq.fastqz
        gzipped: true


#### Write failed reads to a file to save?

Reads are considered failed if they do not pass all output filters.
If so, they original input read is written, in FASTQ format, to a path 
specified with the option `--failed` or `-f`. No path, no output.
Or, use the YAML key `failed:` in the `output:` block. Ex:

    itermae --failed failed.fastq [...]

or in the config file

    output:
        failed: failed.fastq

#### Write a report with per-read pass/fail decisions and match statistics?

While debugging and tuning arguments, it can be useful to have a detailed
per-read report of match statistics. This is recommended to only do on
a small subset of the data, since this is a large (per read) output.

To generate this, specify a path in which to generate a report CSV file
using the `--report` or `-r` arguments. Or, use the YAML key `report:` 
in the `output:` block. Ex:

    head -n 1000 itermae/data/barseq.fastq | itermae --report report.csv [...]

or in the config file

    output:
        report: report.csv

### Matches  <a id="matches"/>

Matches are the heart of the tool. `itermae` simply uses the `regex` library,
and is essentially a utility to arrange `regex` fuzzy expression matches on
`Biopython SeqIO` objects such that the groups are handled well and
filtered/formatted appropriately for amplicon sequencing analyses.

However, exposing this flexibility makes its use complicated. There are two
ways of defining it - the YAML method may be simplier if you're not used to
regular expressions.

- With command-line :

  This can be specified on the command-line as one match per each argument
  of `-m` or `--match`. This requires that you write a regular expression
  such that capture groups are named using the 
  `(?P<nameOfCaptureGroup>ATCGgroupPatternATCG)` syntax, and can use the
  `regex` fuzzy matching denotations such as, for example `{e<=2}` to 
  permit 2 or fewer errors in the preceeding match group. 
  
  See examples, or [`regex` package documentation](https://pypi.org/project/regex/)
  for the details and authoritative documentation!
  
        
- With YAML config file:
    
  Alternatively, the YML configuration mode offers a simpler although more
  bulky syntax. Inside a YML configuration file, you specify a dictionary
  with the `matches:` top-level key. Inside this, using indented `-`'s,
  you specify an array of matches. For each, they need a value behind the
  key `use:` to denote what is matched against (by default `input`). These
  can groups captured from previous matches. Here's an example:
  
    matches:
      -   use: input
          pattern: NGTCCACGAGGTCTCTNCGTACGCTG 
          marking: ABBBBBBBBBBBBBBBCDDDDDDDDD 
          marked_groups:
              A:
                  name: sampleIndex
                  repeat: 5
              B:
                  name: prefix
                  allowed_errors: 2
              C:
                  name: barcode
                  repeat_min: 18
                  repeat_max: 22
              D:  
                  allowed_insertions: 1
                  allowed_deletions: 2
                  allowed_substititions: 2
  
  Inside each match, 
  you specify the pattern to match behind the key `pattern:`. This
  recognizes IUPAC codes, so `A` is `A`, while `R` matches `A` or `G`.
  Additionally `*` matches zero or more of anything, while `+` matches one 
  or more of anything.
  
  For each character of the pattern, you also need to specify a `marking:`
  string specifying what group each character belongs to. This must be the
  same length as the `pattern:`, and each different character denotes a 
  different group.
  
  See the above example, it
  denotes a pattern of group `A` with any base, group `B` with a specific
  sequence, group `C` with any base, and group `D` with a specific sequence.
  
  The in a key `marked_groups:` you can specify for each of these groups
  what the parameters of the match are. Specifically, you can give the
  group a `name:` when matched, you can denote that the group must `repeat:`
  a certain number of times or a `repeat_min:` and `repeat_max:` number of
  times. You can set permissible errors with `allowed_errors:`, or specify
  this for specific types of errors - `allowed_insertions:`, 
  `allowed_deletions:`, `allowed_substitutions:`.
  
  Thus, this is roughly equivalent to the fuzzy regular expression:
  
      (?P<sampleIndex>[ATCGN]{5,5})(?P<prefix>GTCCACGAGGTCTCT){e<=2}(?P<barcode>[ATCGN]{18,22})(?P<untitled_1>CGTACGCTG){i<=1,d<=2,s<=2}'

  Note that it is broken onto multiple lines due to length.
  
  
Then there's the regular expression. These are complex, and `itermae` is designed to expose this complexity 
to the user directly so that you can do more with the tool! Here, I will try to explain a cookbook/copy-paste 
version that you can use if you're new to regexes and need to trial-and-error the result you want:

The pattern you define is matched against the read. So the pattern `.` will match one letter of anything, 
which is why I used it in the above examples (it always matches!). Putting an `ATCG` will only match when
there's exactly a `ATCG` in the read, anywhere in the read, but exactly those four letters in that sequence.
If you want to only one character of DNA, you can use `[ATCG]` to match any one of those four letters. 
*Always use uppercase for `itermae`!* If you want to include `N`, you can do that with `[ATCGN]`. 

To match two letters, you could write `[ATCGN][ATCGN]`, or more easily `[ATCGN]{2,2}`. 
The `{}` braces help you specify how many to match. 
You can specify one number `{2,2}`, or a range `{2,5}`.
So for a DNA barcode of length 18-22 bases, you may want to specify `[ATCGN]{18,22}`.

`(` `)` parentheses are used to denote groups, like they're used in math. 
Capture groups use these `()` but also use `?P` and `<>` brackets to name the group. 
A named capture group uses notation like `(?P<barcode>[ATCGN]{18,22})`, where 
`(?P<` is the start, `barcode` is the name of the group, and the group is whatever 
matches to `[ATCGN]{18,22}`, until the `)`. It's complex syntax, but necessary to clearly
denote what is the name of the group and what is the group.

To match this barcode only when it is between defined sequences, for example priming sites, you can use a more
complex pattern like `GGTCTCT(?P<barcode>[ATCGN]{18,22})CGTACGCT`. This matches a sequence where:
- There is a `GGTCTCT`
- then 18 to 22 bases of either A, T, C, G, or N
- then `CGTACGCT`

Crucially, it then saves that center 18 to 22 bases as a group named `barcode`.
That means you can then only output that group, or you can put it into the output ID, or you can even match
other operation patterns against _just that group_.

For example (using a `\` to continue it onto a new line):

#### fuzzy-ness in regular expressions

Fuzzy means error-tolerant. 
However, errors are common in sequencing. Substitutions, whether they come from a mutation, PCR misincorporation,
or erroneous basecall can interfere with your ability to use defined sequence to parse your read.
Indels are rarer in Illumina sequencing, but any indel early will affect the position of the later sequence.
Barcode/amplicon libraries also have size heterogeneity from construction and propogation.

To handle this, we can use "fuzzy" regular expressions. These are regular expressions, but they're approximate.
This can tolerate errors of single-base changes or errors in length. `itermae` uses the `regex` module to do
fuzzy regular expression matching. This is an excellent library that provides a new syntax to specify
"fuziness" of a group.

To write the above example in another way, we add after the group the modifier `{e<=1}`. 
This means that less-than-or-equal to 1 errors of any kind (insertion, deletion, substitution) are tolerated
in the previous group.
So we can change our operation to `input > (GGTCTCT){e<=1}(?P<barcode>[ATCGN]{18,22})"`, and this will
tolerate one or fewer errors in the `GGTCTCT` just in front of the barcode. Note the parentheses are necessary.

## Multiple operations

You can add multiple operations. Each sequential operation added on in the command line is executed
one after another. This allows you to take one capture group and search within it for more groups.
The big advantage of this is speed (less combinatorial complexity of mismatches and degeneracy)
and less complicated regular expressions.

For example, here's the above example but broken into two operations. The first takes the sample index, the
second takes the barcode. Note that I introduce using `*` to mean 0 or more (usually much more) of `[ATCGN]`.
Also, I put the `CTCCACGAG` group, with an error `{e<=1}` expression, inside the `rest` capture group.

In [ ]:
cat itermae/data/toy.fastq | itermae \
        -o "input > (?P<sampleIndex>[ATCGN]{5,5})(?P<rest>(GTCCACGAG){e<=1}[ATCGN]*)" \
        -o "rest  > (GTCCACGAGGTCTCT){e<=2}(?P<barcode>[ATCGN]{18,22})(CGTACGCTG){e<=1}" \
        -oseq "barcode" -oid "input.id+\"_\"+sampleIndex.seq" -of "fasta"

Note that you can also use multiple outputs. For example, here I'll output the barcode with and without
the priming sites around it.

In [ ]:
cat itermae/data/toy.fastq | itermae \
        -o "input > (?P<sampleIndex>[ATCGN]{5,5})(?P<rest>(GTCCACGAG){e<=1}[ATCGN]*)" \
        -o "rest  > (?P<upPrime>GTCCACGAGGTCTCT){e<=2}(?P<barcode>[ATCGN]{18,22})(?P<downPrime>CGTACGCTG){e<=1}" \
        -oseq "barcode" -oid "input.id+\"_\"+sampleIndex.seq" \
        -oseq "upPrime+barcode+downPrime" -oid "input.id+\"_withFixedFlanking_\"+sampleIndex.seq" \
        -of "fasta"

Doesn't that look nicer? And it runs a bit faster, since you don't use combinations of searching for all
possible errors if the from and back parts, especially with complex parsing schemes.

In [ ]:
cat itermae/data/toy.fastq \
    | itermae -o "input > (GGTCTCT){e<=1}(?P<barcode>[ATCGN]{18,22})CGTACG" -oseq "barcode" -of "fasta"

In [ ]:
cat itermae/data/toy.fastq \
    | itermae -o "input > (?P<fixed>GGTCTCT){e<=1}(?P<barcode>[ATCGN]{18,22})" -oseq "fixed" -of "fasta"

So there's a `GGTCTAT` in there, the second one. 

I can also append this group to the input ID, and then have the barcode be
the main output sequence again. Note that changing the ID means you have to access attributes like 
`.id` or `.seq` from each captured group, as they're handled like BioPython SeqRecords internally. 
And the below also puts a `+'_'+` between them for readability.

In [ ]:
cat itermae/data/toy.fastq \
    | itermae -o "input > (?P<fixed>GGTCTCT){e<=1}(?P<barcode>[ATCGN]{18,22})" \
        -oseq "barcode" -oid "input.id+'_'+fixed.seq" -of "fasta"

You can build on this to make it more complex. These particular reads actually have a five base index 
at the beginning that denotes the sample, so below I'll extract that and append it to the read ID, then output
the barcode, while fuzzy matching the priming sites.

In [ ]:
cat itermae/data/toy.fastq \
    | itermae \
        -o "input > (?P<sampleIndex>[ATCGN]{5,5})(GTCCACGAGGTCTCT){e<=2}(?P<barcode>[ATCGN]{18,22})(CGTACGCTG){e<=1}" \
        -oseq "barcode" -oid "input.id+'_'+sampleIndex.seq" -of "fasta"

This should get you going, but regular expressions are a powerful and common feature in programming and 
analysis. For more, I recommend you search for "python regular expression tutorial" 
or the like and read a few articles that talk about this, then consult the `regex` package documentation 
for more information about those specific modifiers.

### Output filtering and forming

What to output to the output?


- With command-line :

  This can be specified on the command-line as one match per each argument
  of `-m` or `--match`. This requires that you write a regular expression
  such that capture groups are named using the 
  

#### filters
A more useful example would be filtering on quality. Those are accessible by accessing the
`.quality` attribute of the sequence in question. 

You can also access other attributes of each group using
that name (the group name).
( Internally, this references an object with attributes of `start`, `end`,
`length`, `quality`, and `seq`.)
You can use these together in filters. You can't filter on the raw `input`,
(or on `dummyspacer` for that matter).
See `apply_filter()` function in the `__init__.py` file for details.

To make building these easier, the `statistics` package is loaded. 
So you can do things like filtering on
`min(barcode.quality) >= 33` or `statistics.median(barcode.quality) > 34 and barcode.length >= 16`.

For debugging, you can print the thing you're filtering and see that spit
out. So something like `print(statistics.median(barcode.quality))`
(this is using `eval` directly by the way, so be responsible).

In [ ]:
cat itermae/data/toy.fastq \
    | itermae \
        -o "input > (?P<sampleIndex>[ATCGN]{5,5})(GTCCACGAGGTCTCT){e<=2}(?P<barcode>[ATCGN]{18,22})(CGTACGCTG){e<=1}" \
        -oseq "barcode" -oid "input.id+'_'+sampleIndex.seq" -of "fasta" \
        --filter "barcode.start == 20 and barcode.length == 20"

In [ ]:
cat itermae/data/toy.fastq \
    | itermae \
        -o "input > (?P<sampleIndex>[ATCGN]{5,5})(GTCCACGAGGTCTCT){e<=2}(?P<barcode>[ATCGN]{18,22})(CGTACGCTG){e<=1}" \
        -oseq "barcode" -oid "input.id+'_'+sampleIndex.seq" -of "fastq" \
        --filter "statistics.median(barcode.quality) >= 34 and min(barcode.quality) < 36"

## Debugging

A lot of what `itermae` is doing depends on the regular expressions, so it can get a bit complex and debugging
effectively is important. One strategy is to take a sample of reads
from the middle of the file (or at least not the first ~100,000 of a FASTQ file) and feed in a hundred or so reads
at a time to see how it works.

To help instrument how the operations you define are working, there's a `--report` feature.
This argument specifies a file path where `itermae` will generate a report for each read, reporting if it was
successful or not, what the input read was, what filters it passed and where it matched.

There's also a `--failed` argument, a path where `itermae` puts the failed reads (for whatever reason)
for later inspection.

Additionally, there's `-v` or `--verbose` arguments. One outputs information about the run to standard error
(so that standard output still works fine for piping output), and additional `-v`'s added on increase the level
of reporting to individual reads and individual operations.

Here's an example with the first level of verbosity, and outputing a report and failed reads file.

In [ ]:
cat itermae/data/toy.fastq | itermae \
        -o "input > (?P<sampleIndex>[ATCGN]{5,5})(GTCCACGAGGTCTCT){e<=2}(?P<barcode>[ATCGN]{18,22})(CGTACGC){e<=1}" \
        -oseq "barcode" -oid "input.id+'_'+sampleIndex.seq" -of "sam" \
        --report report.csv --failed failed.fastq -v \
    > out.sam

head out.sam
rm out.sam

echo "The report:"
head -n 4 report.csv
rm report.csv
echo
echo "The first of the failed reads:"
head -n 4 failed.fastq
rm failed.fastq

We see the third record (at fake positions 10002) failed in not being able to for the output.
We see the failed read, and see that it's lacking the pattern we're looking for.

For the report CSV, the columns are:

- passed or not
- input ID
- input sequence
- output sequence, if successful
- the result of any filters (later), which here is just True
- a string that denotes each group matched, where it starts, ends, and length

## Parallel chopping, intended usage

### Input/output

`itermae` is based on a previous tool, but this implementation has removed any pretense of trying to
handle parallelization internally. Instead, this tool just takes a stream of standard in FASTQ and parses it.
You can readily parallelize by using the command-line tool GNU `parallel` (yes I'll cite it).
This designs limits complexity of code and prevents a certain _fuzzy_ memory leak...

However, there is a single-thread ability to read from a file on disk using `--input`, and that can be indicated
as being gzipped with the flag `-z`. Similarly, `itermae` is designed to output to standard output, but you can
specify an output file with `--output`. 

Below is an example of using `parallel` to parallelize the above example, 
of course you'll need that installed and available:

In [ ]:
cat itermae/data/toy.fastq \
    | parallel --pipe -L 4 \
        'itermae -o "input > (?P<sampleIndex>[ATCGN]{5,5})(GTCCACGAGGTCTCT){e<=2}(?P<barcode>[ATCGN]{18,22})(CGTACGCTG){e<=1}" \
            -oseq "barcode" -oid "input.id+\"_\"+sampleIndex.seq" -of "fasta" ' 

The `--pipe` arguement specifies to pipe the input into `itermae`, and do that in chunks of `-L` size.
You will probably want to use a chunk size of 10-100 thousand, but do that in multiples of 4 (since it's a fastq)!
So something like `parallel --pipe -L 40000` is what you'd want to use on real data.

Then you put the `itermae` command in quotes. Here I've used `'` on the outside. Note that if I need to `"`
something inside an argument building the output sequence, I need to escape that with a `\`.

You can also restrict the jobs to run with arguments like `-j`, so consult the `parallel` manual for more info.
I usually leave it to figure things out for me, and it makes full use of the machine.

80character ruler
         X         X         X         X         X         X         X         X